# Evaluating Response Quality Multi Models

Set up and configure multiple Azure OpenAI model endpoints.


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv('../../.env')

env_var = {
    "gpt-4o": {
        "endpoint": os.environ.get("AOAI_GPT4O_ENDPOINT"),
        "key": os.environ.get("AZURE_OPENAI_API_KEY"),
    },
    "gpt-4o-mini": {
        "endpoint": os.environ.get("AOAI_GPT4O_MINI_ENDPOINT"),
        "key": os.environ.get("AZURE_OPENAI_API_KEY"),
    },    
}

ai_project_endpoint=os.environ["AI_PROJECT_ENDPOINT"]

Display the evaluation dataset.

In [ ]:
import pandas as pd

df = pd.read_json("evaluation_dataset.jsonl", lines=True)
print(df.head())

Initiate the model config

In [ ]:
from azure.ai.evaluation import AzureOpenAIModelConfiguration

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
)

Initiate and run automated evaluations comparing model outputs.

In [ ]:
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import QAEvaluator
from model_endpoints import ModelEndpoints
import random

qa_evaluator = QAEvaluator(model_config)

models = [
    "gpt-4o",
    "gpt-4o-mini",
]

path = "./evaluation_dataset.jsonl"
randomNum = random.randint(1111, 9999)

for model in models:
    results = evaluate(
        evaluation_name="Eval-Run" + str(randomNum) + "-" + model.title(),
        data=path,
        target=ModelEndpoints(env_var, model),
        evaluators={
            "qa": qa_evaluator,
        },
        azure_ai_project=ai_project_endpoint,
        evaluator_config={
            "qa": {
                "column_mapping": {
                    "query": "${data.query}",
                    "context": "${data.context}",
                    "ground_truth": "${data.ground_truth}",
                    "response": "${target.response}",
                },
            },
        },
    )

# Use the URL to inspect the results on the UI.
print(f'AI Foundry URL: {results.get("studio_url")}')

In [ ]:
pd.DataFrame(results["rows"])